<a href="https://colab.research.google.com/github/DJNahata/Style-GANS-Practice/blob/master/Discriminator_Network_in_Nnabla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install nnabla

     |████████████████████████████████| 14.2MB 235kB/s 
     |████████████████████████████████| 4.8MB 44.5MB/s 


In [0]:
import numpy as np
import nnabla as nn
import nnabla.parametric_functions as PF
import nnabla.functions as F
import nnabla.solvers as S
import nnabla.logger as logger
import nnabla.utils.save as save
from nnabla.parameter import get_parameter_or_create
from nnabla.ext_utils import get_extension_context
from nnabla.parametric_functions import parametric_function_api
import nnabla.initializer as I
nn.set_auto_forward(True)
print("DJ1")

2020-04-21 17:43:12,586 [nnabla][INFO]: Initializing CPU extension...


DJ1


In [0]:
from __future__ import absolute_import
from six.moves import range
print("DJ1")

import functools
# Globally prepared params.
w = nn.parameter.get_parameter_or_create('BLURRING_FILTER', shape=(3,3), need_grad=False)  #why and how is it used for the parameter
w.d = np.array([[1., 2., 1.], [2., 4., 2.], [1., 2., 1.]])   

print('')


def lerp(a, b, t):
  if(t>=0):
    return ((a + (b - a) * t))
  else:
    return a

def lerp_clip(a, b, t):
    return (a + (b - a) * F.clip_by_value(t, 0.0, 1.0))


#Computing the mean across the feature maps i.e. axis 2 and 3 for H and W with data format of x as NCHW
#--------------------
def compute_channel_mean(x, keepdims = True):
    ch_mean = F.mean(x, axis = [2,3], keepdims = keepdims)
    return ch_mean

#--------------------


#-------------------------------
#Computing the standard deviation across the feature maps i.e. axis 2 and 3 for H and W with data format of x as NCHW
def compute_channel_std(x, keepdims = True):
    x1 = compute_channel_mean(x)
    var = F.mean(F.pow_scalar((x - x1), 2.0), axis = [2,3], keepdims = keepdims)
    std = F.pow_scalar(var, 0.5)
    return std

#-------------------------------


#---------------------------------------------------
#Adaptive Instance Normalization to be used
def adain(x, y):
    y_s, y_b = y
    ch1_mean = compute_channel_mean(x)
    ch1_std = compute_channel_std(x)
    x = (x - ch1_mean) / ch1_std
    a1 = (y_s * x) + y_b
    return a1 

#-------------------------------


#New blur function to be used ------------------------------------------
def blur2d(x, f=[1,2,1], normalize = True, flip = False, stride = 1):
    ## assert x.ndim == 4 and all(dim.value is not None for dim in x.shape[1:])
    assert isinstance(stride, int) and stride >= 1

    f = np.array(f, dtype = np.float32)

    if f.ndim == 1:
        f = f[:, np.newaxis] * f[np.newaxis, :]

    if flip:
        f = f[::-1, ::-1]

    if normalize:
        f = f / np.sum(f)

    f = f[ np.newaxis, :, :]
    f = np.tile(f, [int(x.shape[1]), 1, 1])
    if f.shape == (1, 1) and f[0,0] == 1:
        return x

    #orig_dtype = x.dtype
    strides = (stride, stride) 
    w = nn.Variable.from_numpy_array(f)
    padding = (w.shape[1] // 2, w.shape[2] // 2)

    y = F.depthwise_convolution(x, w, pad = padding, stride = strides)
    return y

DJ1



In [0]:
def get_weight(he_std, gain = np.sqrt(2), use_wscale = False, lrmul = 1):
    if use_wscale:
        init_std = 1.0 / lrmul
        runtime_coeff = he_std * lrmul
    else: 
        init_std = he_std / lrmul
        runtime_coeff = lrmul

    w_init = I.NormalInitializer(sigma = init_std)   #I.NormalInitializer function is used with sigma argument as the standard deviation, so as to get the weights of these variables 
    return w_init, runtime_coeff

In [0]:
def downscale2d(x, factor = 2, gain = 1):
    #assert x.ndim == 4 and all(dim.value is not None for dim in x.shape[1:])
    assert isinstance(factor, int) and factor >=1

    if factor == 2:
        f = [np.sqrt(gain) / factor] * factor
        f = np.array(f, dtype = np.float32)
        if f.ndim == 1:
            f = f[:, np.newaxis] * f[np.newaxis, :]
        assert f.ndim == 2

        f = f[np.newaxis, :, :]
        f = np.tile(f, [int(x.shape[1]), 1, 1])
        w = nn.Variable.from_numpy_array(f)
        strides = (factor, factor)
        padding = (w.shape[1] // 2, w.shape[2] // 2)
        
        y = F.depthwise_convolution(x, w, stride = strides)
        return y 
    
    if gain != 1:
        x = x * gain

    if factor == 1:
        return x

    kernel = [factor, factor]
    y = F.average_pooling(x, kernel = kernel, stride = kernel)    # a bit of doubt regarding this statement
    return y

In [0]:
def dense(x, fmaps, gain = np.sqrt(2), use_wscale = False, lrmul = 1, **kwargs):
    if len(x.shape) > 2:
        x = F.reshape(x, [-1, np.prod([d for d in x.shape[1:]])])
    he_std = I.calc_normal_std_he_forward(x.shape[1], fmaps)
    w_init, runtime_coeff = get_weight(he_std, gain, use_wscale, lrmul, **kwargs)
    w = nn.parameter.get_parameter_or_create('Dense', shape = (x.shape[1], fmaps), initializer = w_init, need_grad = True)
    w = w * runtime_coeff
    return (F.affine(x, w, bias=None, base_axis=1))

In [0]:
#Upscale operation function to be used 
def upscale2d(x, factor = 2, gain = 1):
    # to check whether the shape of the x is 4 or not
    #assert x.ndim == 4 and all(dim.value is not None for dim in x.shape[1:])
    assert isinstance(factor, int) and factor >=1 

    if gain!=1:
        x = x * gain
    
    if factor == 1:
        return x

    s = x.shape
    #reshape and tile function similar as that of the tensorflow
    x = F.reshape(x, [-1, s[1], s[2], 1, s[3], 1])
    x = F.tile(x, [1, 1, 1, factor, 1, factor])
    x = F.reshape(x, [-1, s[1], s[2] * factor, s[3] * factor])
    return x

In [0]:
def conv2d(x, fmaps, kernel, gain = np.sqrt(2), use_wscale = False, **kwargs):
  with nn.parameter_scope('Convolution'):
    assert isinstance(kernel, int) or isinstance(kernel, float)
    assert kernel >= 1 and kernel % 2 == 1
    if kernel == 1:
        padding = None
    else:
        padding = ((kernel - 2), (kernel - 2))  #tuple of padding
    kernel_tuple = (kernel, kernel)  #tuple of kernel_tuple
    he_std = I.calc_normal_std_he_forward(fmaps, x.shape[1], kernel_tuple)        # (kernel, kernel, x.shape[1], fmaps) (3, 2, 0, 1)
    w_init, runtime_coeff = get_weight(he_std, gain, use_wscale, **kwargs)
    return PF.convolution(x, fmaps, kernel = kernel_tuple, w_init = w_init, with_bias = False, pad = padding, apply_w = lambda w: w * runtime_coeff)

#usage of the Noise application which is used in the synthesis network.
def apply_noise(x, noise_var = None, randomize_noise = True):
    assert len(x.shape) == 4
    with nn.parameter_scope('Noise'):
        if noise_var is None or randomize_noise:
            noise = F.randn(shape = (x.shape[0], 1, x.shape[2], x.shape[3]))
        else:
            print("Dj1")
            noise = noise_var
        weight = nn.parameter.get_parameter_or_create(name = 'weight', shape = [x.shape[1]], initializer=I.ConstantInitializer(0))     # For Creating Random nnabla variable such that each entry follows the Normal Distribution..
        weight = F.reshape(weight, [1, -1, 1, 1], inplace= False)
        return (x + noise * weight)

In [0]:
def upscale2d_conv2d(x, fmaps, kernel, gain = np.sqrt(2), use_wscale = False, fused_scale = 'auto', **kwargs):
    assert kernel >= 1 and kernel % 2 == 1
    assert fused_scale in [True, False, 'auto']
    if fused_scale == 'auto':
        fused_scale = min(x.shape[2:]) * 2 >= 128

    if not fused_scale:
        return conv2d(upscale2d(x), fmaps, kernel, **kwargs)

    kernel_tuple = (kernel, kernel)
    he_std = I.calc_normal_std_he_forward(x.shape[1], fmaps, kernel_tuple)        # (kernel, kernel, x.shape[1], fmaps)  (2, 3, 0, 1)
    w_init, runtime_coeff = get_weight(he_std, gain, use_wscale, **kwargs)
    w = nn.parameter.get_parameter_or_create('deconv/W', shape = (x.shape[1], fmaps, kernel, kernel), initializer = w_init, need_grad = True)
    w = w * runtime_coeff
    w = F.pad(w, (1, 1, 1, 1), mode = 'constant')
    w = w[..., 1:, 1:] + w[..., :-1, 1:] + w[..., 1:, :-1] + w[..., :-1, :-1]
    kernel_size = w.shape[-1]
    padding = ((kernel_size - 2) / 2, (kernel_size - 2) / 2)
    stride = (2 , 2)
    return F.deconvolution(x, w, bias = None, pad = padding, stride = stride)

In [0]:
# Change in the apply_bias function.
def apply_bias(x, lrmul = 1):
    b = nn.parameter.get_parameter_or_create(name = 'bias', shape = [x.shape[1]], initializer = I.ConstantInitializer(0))   # Change the initializer to either float or convert the datatype to float32.
    b = b * lrmul  # No Modifications requried..
    if x.ndim == 2:
        return (x + F.reshape(b, [1, -1]))
    return (x + F.reshape(b, [1, -1, 1, 1])) 

In [0]:
#defining the style modulation i.e. combining ys and yb with the resultant of the addition of the noise and the feature maps
def style_mod(x, dlatent, **kwargs):
    with nn.parameter_scope('StyleMod'):
        style = apply_bias(dense(dlatent, fmaps = x.shape[1] * 2, **kwargs))
        style = F.reshape(style, [-1, 2, x.shape[1]] + [1] * (len(x.shape) - 2))
        return x * (style[:,0] + 1) + style[:,1]


In [0]:
def conv2d_downscale2d(x, fmaps, kernel, gain = np.sqrt(2), use_wscale = False, fused_scale = 'auto', **kwargs):
    #assert kernel % 2 == 1 and kernel >= 1
    assert fused_scale in [True, False, 'auto']
    if fused_scale == 'auto':
        fused_scale = min(x.shape[2:]) >= 128   #why is this written and what is the condition behind that

    if not fused_scale:
        return downscale2d(conv2d(x, fmaps, kernel, **kwargs))

    kernel_tuple = (kernel, kernel)
    he_std = I.calc_normal_std_he_forward(fmaps, x.shape[1], kernel_tuple)       # (kernel, kernel, x.shape[1], fmaps) (3, 2, 0, 1)
    w_init, runtime_coeff = get_weight(he_std, gain = gain, use_wscale = use_wscale, **kwargs)
    w = nn.parameter.get_parameter_or_create('conv/W', shape = (fmaps, x.shape[1], kernel, kernel), initializer = w_init, need_grad = True)
    w = w * runtime_coeff
    w = F.pad(w, (1, 1, 1, 1), mode = 'constant')
    w = w[..., 1:, 1:] + w[..., :-1, 1:] + w[..., 1:, :-1] + w[..., :-1, :-1]
    w = w * 0.25
    kernel_size = w.shape[-1]
    padding = ((kernel_size - 2) / 2, (kernel_size - 2) / 2)
    stride = (2,2)
    return F.convolution(x, w, bias=None, pad = padding, stride = stride)

In [0]:
def minibatch_stddev_layer(x, group_size=4, num_new_features=1):
    group_size = min(group_size, x.shape[0])     # Minibatch must be divisible by (or smaller than) group_size.
    s = x.shape                                             # [NCHW]  Input shape.
    y = F.reshape(x, [group_size, -1, num_new_features, s[1]//num_new_features, s[2], s[3]])   # [GMncHW] Split minibatch into M groups of size G. Split channels into n channel groups c.
    y -= F.mean(y, axis=0, keepdims=True)           # [GMncHW] Subtract mean over group.
    y = F.mean(F.pow_scalar(y, 2.0), axis=0)                # [MncHW]  Calc variance over group.
    y = F.add_scalar(F.pow_scalar(y, 2.0) + 1e-8)                                   # [MncHW]  Calc stddev over group.
    y = F.mean(y, axis=[2,3,4], keepdims=True)      # [Mn111]  Take average over fmaps and pixels.
    y = F.mean(y, axis=[2])                         # [Mn11] Split channels into c channel groups
    y = F.tile(y, [group_size, 1, s[2], s[3]])             # [NnHW]  Replicate over group and pixels.
    return F.concatenate(x, y, axis=1)

In [0]:
#--------------------------------------
#application of the activation function
def leaky_relu(x):
    return functools.partial(F.leaky_relu, alpha = 0.2)(x)
#--------------------------------------


In [0]:
def pixel_norm(x, epsilon = 1e-8):
    mean1 = F.mean(F.pow_scalar(x, 2.0), axis = 1, keepdims = True)
    new_mean1 = F.add_scalar(mean1, epsilon)
    denominator = F.pow_scalar(new_mean1, 0.5)
    x = F.div2(x, denominator)
    return x


In [0]:
def instance_norm(x, epsilon = 1e-8):
    assert len(x.shape) == 4
    with nn.parameter_scope('InstanceNorm'):
        x1 = compute_channel_mean(x)
        x = x - x1
        x2 = compute_channel_mean(F.add_scalar(F.pow_scalar(x , 2.0), epsilon))
        x2 = F.pow_scalar(x2, 0.5)
        x = x / x2
        return x

In [0]:
def D_basic(
    images_in,                          # First input: Images [minibatch, channel, height, width].
    #labels_in,                          # Second input: Labels [minibatch, label_size].
    num_channels        = 3,            # Number of input color channels. Overridden based on dataset.
    resolution          = 256,           # Input resolution. Overridden based on dataset.
    label_size          = 0,            # Dimensionality of the labels, 0 if no labels. Overridden based on dataset.
    fmap_base           = 8192,         # Overall multiplier for the number of feature maps.
    fmap_decay          = 1.0,          # log2 feature map reduction when doubling the resolution.
    fmap_max            = 512,          # Maximum number of feature maps in any layer.
    nonlinearity        = 'lrelu',      # Activation function: 'relu', 'lrelu',
    use_wscale          = True,         # Enable equalized learning rate?
    mbstd_group_size    = 4,            # Group size for the minibatch standard deviation layer, 0 = disable.
    mbstd_num_features  = 1,            # Number of features for the minibatch standard deviation layer.
    dtype               = 'float32',    # Data type to use for activations and outputs.
    fused_scale         = 'auto',       # True = fused convolution + scaling, False = separate ops, 'auto' = decide automatically.
    blur_filter         = [1,2,1],      # Low-pass filter to apply when resampling activations. None = no filtering.
    structure           = 'linear',       # 'fixed' = no progressive growing, 'linear' = human-readable, 'recursive' = efficient, 'auto' = select automatically.
    is_template_graph   = False,        # True = template graph constructed by the Network class, False = actual evaluation.
    **_kwargs):                         # Ignore unrecognized keyword args.

    resolution_log2 = int(np.log2(resolution))
    assert resolution == 2**resolution_log2 and resolution >= 4

    def nf(stage): return min(int(fmap_base / (2.0 ** (stage * fmap_decay))), fmap_max)

    def blur(x):
        return blur2d(x, blur_filter) if blur_filter else x

    if structure == 'auto':
        structure = 'linear' if is_template_graph else 'recursive'

    act, gain = {'relu': (F.relu, np.sqrt(2)), 'lrelu': (leaky_relu, np.sqrt(2))}[nonlinearity]

    #lod_in = tf.get_variable('lod', initializer=np.float32(0.0), trainable=False)
    lod_in = 0
    scores_out = None

    # Building blocks.
    def fromrgb(x, res):  # res = 2..resolution_log2
        with nn.parameter_scope('FromRGB_lod{}'.format(resolution_log2 - res)):
            return act(apply_bias(conv2d(x, fmaps=nf(res-1), kernel=1, gain=gain, use_wscale=use_wscale)))

    def block(x, res):  # res = 2..resolution_log2
        with nn.parameter_scope('{}x{}'.format(2**res, 2**res)):
            if res >= 3:  # 8x8 and up
                with nn.parameter_scope('Conv0'):
                    x = act(apply_bias(conv2d(x, fmaps=nf(res-1), kernel=3, gain=gain, use_wscale=use_wscale)))
                with nn.parameter_scope('Conv1_down'):
                    x = act(apply_bias(conv2d_downscale2d(blur(x), fmaps=nf(res-2), kernel=3, gain=gain, use_wscale=use_wscale, fused_scale=fused_scale)))
            else:  # 4x4
                if mbstd_group_size > 1:
                    x = minibatch_stddev_layer(x, mbstd_group_size, mbstd_num_features)
                with nn.parameter_scope('Conv'):
                    x = act(apply_bias(conv2d(x, fmaps=nf(res-1), kernel=3, gain=gain, use_wscale=use_wscale)))
                with nn.parameter_scope('Dense0'):
                    x = act(apply_bias(dense(x, fmaps=nf(res-2), gain=gain, use_wscale=use_wscale)))
                with nn.parameter_scope('Dense1'):
                    x = apply_bias(dense(x, fmaps=max(label_size, 1), gain=1, use_wscale=use_wscale))
            return x

    # Fixed structure: simple and efficient, but does not support progressive growing.
    if structure == 'fixed':
        x = fromrgb(images_in, resolution_log2)
        for res in range(resolution_log2, 2, -1):
            x = block(x, res)
        scores_out = block(x, 2)

    
    # Linear structure: simple but inefficient.
    if structure == 'linear':
        img = images_in
        x = fromrgb(img, resolution_log2)
        for res in range(resolution_log2, 2, -1):
            lod = resolution_log2 - res
            x = block(x, res)
            img = downscale2d(img)
            y = fromrgb(img, res - 1)
            with nn.parameter_scope('Grow_lod%d' % lod):
                x = lerp(x, y, lod_in - lod)
        scores_out = block(x, 2)

    # Recursive structure: complex but efficient.
    if structure == 'recursive':
        def cset(cur_lambda, new_cond, new_lambda):
            return lambda: tf.cond(new_cond, new_lambda, cur_lambda)
        def grow(res, lod):
            x = lambda: fromrgb(downscale2d(images_in, 2**lod), res)
            if lod > 0: x = cset(x, (lod_in < lod), lambda: grow(res + 1, lod - 1))
            x = block(x(), res); y = lambda: x
            if res > 2: y = cset(y, (lod_in > lod), lambda: tflib.lerp(x, fromrgb(downscale2d(images_in, 2**(lod+1)), res - 1), lod_in - lod))
            return y()
        scores_out = grow(2, resolution_log2 - 2)
    

    # Label conditioning from "Which Training Methods for GANs do actually Converge?"
    if label_size:
        with nn.parameter_scope('LabelSwitch'):
            scores_out = F.sum(scores_out * labels_in, axis=1, keepdims=True)

    scores_out = F.identity(scores_out)
    return scores_out

In [0]:
i1 = np.random.random((1,3,256,256))
i2 = nn.Variable.from_numpy_array(i1)

In [0]:
output1 = D_basic(i2, use_wscale = False)

In [0]:
print(output1.shape)

(1, 1)


In [0]:
import pickle
with open("we1.txt", "rb") as fp:   
   we1 = pickle.load(fp)

In [0]:
with open("we2.txt", "rb") as fp:   
   we2 = pickle.load(fp)

In [0]:
with open("we3.txt", "rb") as fp:   
   we3 = pickle.load(fp)

In [0]:
with open("we4.txt", "rb") as fp:   
   we4 = pickle.load(fp)

In [0]:
with open("we5.txt", "rb") as fp:   
   we5 = pickle.load(fp)

In [0]:
with open("we6.txt", "rb") as fp:   
   we6 = pickle.load(fp)

In [0]:
with open("we7.txt", "rb") as fp:   
   we7 = pickle.load(fp)

In [0]:
ax = nn.get_parameters()

In [0]:
l = 0
for i, j in ax.items():
    print(i)

FromRGB_lod0/Convolution/conv/W
FromRGB_lod0/bias
256x256/Conv0/Convolution/conv/W
256x256/Conv0/bias
256x256/Conv1_down/conv/W
256x256/Conv1_down/bias
FromRGB_lod1/Convolution/conv/W
FromRGB_lod1/bias
128x128/Conv0/Convolution/conv/W
128x128/Conv0/bias
128x128/Conv1_down/conv/W
128x128/Conv1_down/bias
FromRGB_lod2/Convolution/conv/W
FromRGB_lod2/bias
64x64/Conv0/Convolution/conv/W
64x64/Conv0/bias
64x64/Conv1_down/Convolution/conv/W
64x64/Conv1_down/bias
FromRGB_lod3/Convolution/conv/W
FromRGB_lod3/bias
32x32/Conv0/Convolution/conv/W
32x32/Conv0/bias
32x32/Conv1_down/Convolution/conv/W
32x32/Conv1_down/bias
FromRGB_lod4/Convolution/conv/W
FromRGB_lod4/bias
16x16/Conv0/Convolution/conv/W
16x16/Conv0/bias
16x16/Conv1_down/Convolution/conv/W
16x16/Conv1_down/bias
FromRGB_lod5/Convolution/conv/W
FromRGB_lod5/bias
8x8/Conv0/Convolution/conv/W
8x8/Conv0/bias
8x8/Conv1_down/Convolution/conv/W
8x8/Conv1_down/bias
FromRGB_lod6/Convolution/conv/W
FromRGB_lod6/bias
4x4/Conv/Convolution/conv/W
4x

In [0]:
l = 0
l1 = 0
for i, j in ax.items():
    if(l==6):
        break
    if(l>=0):
        if(l%2==0):
            a1 = np.transpose(we1[l1], axes = [3, 2, 0, 1])
            j.data.copy_from(nn.Variable.from_numpy_array(a1).data)
        else:
            j.data.copy_from(nn.Variable.from_numpy_array(we1[l1]).data)
        l1 = l1 + 1
    l = l + 1

In [0]:
l = 0
l1 = 0
for i, j in ax.items():
    if(l==12):
        break
    if(l>=6):
        if(l%2==0):
            a1 = np.transpose(we2[l1], axes = [3, 2, 0, 1])
            j.data.copy_from(nn.Variable.from_numpy_array(a1).data)
        else:
            j.data.copy_from(nn.Variable.from_numpy_array(we2[l1]).data)
        l1 = l1 + 1
    l = l + 1

In [0]:
l = 0
l1 = 0
for i, j in ax.items():
    if(l==18):
        break
    if(l>=12):
        if(l%2==0):
            a1 = np.transpose(we3[l1], axes = [3, 2, 0, 1])
            j.data.copy_from(nn.Variable.from_numpy_array(a1).data)
        else:
            j.data.copy_from(nn.Variable.from_numpy_array(we3[l1]).data)
        l1 = l1 + 1
    l = l + 1

In [0]:
l = 0
l1 = 0
for i, j in ax.items():
    if(l==24):
        break
    if(l>=18):
        if(l%2==0):
            a1 = np.transpose(we4[l1], axes = [3, 2, 0, 1])
            j.data.copy_from(nn.Variable.from_numpy_array(a1).data)
        else:
            j.data.copy_from(nn.Variable.from_numpy_array(we4[l1]).data)
        l1 = l1 + 1
    l = l + 1

In [0]:
l = 0
l1 = 0
for i, j in ax.items():
    if(l==30):
        break
    if(l>=24):
        if(l%2==0):
            a1 = np.transpose(we5[l1], axes = [3, 2, 0, 1])
            j.data.copy_from(nn.Variable.from_numpy_array(a1).data)
        else:
            j.data.copy_from(nn.Variable.from_numpy_array(we5[l1]).data)
        l1 = l1 + 1
    l = l + 1

In [0]:
l = 0
l1 = 0
for i, j in ax.items():
    if(l==36):
        break
    if(l>=30):
        if(l%2==0):
            a1 = np.transpose(we6[l1], axes = [3, 2, 0, 1])
            j.data.copy_from(nn.Variable.from_numpy_array(a1).data)
        else:
            j.data.copy_from(nn.Variable.from_numpy_array(we6[l1]).data)
        l1 = l1 + 1
    l = l + 1

In [0]:
l = 0
l1 = 0
for i, j in ax.items():
    if(l==40):
        break
    if(l>=36):
        if(l%2==0):
            a1 = np.transpose(we7[l1], axes = [3, 2, 0, 1])
            j.data.copy_from(nn.Variable.from_numpy_array(a1).data)
        else:
            j.data.copy_from(nn.Variable.from_numpy_array(we7[l1]).data)
        l1 = l1 + 1
    l = l + 1

In [0]:
l = 0
l1 = 4
for i, j in ax.items():
    if(l==44):
        break
    if(l>=40):
        if(l%2==0):
            a1 = we7[l1]
            print(a1.shape)
            print(j.shape)
            print()
            j.data.copy_from(nn.Variable.from_numpy_array(a1).data)
        else:
            j.data.copy_from(nn.Variable.from_numpy_array(we7[l1]).data)
        l1 = l1 + 1
    l = l + 1

(8192, 512)
(8192, 512)

(512, 1)
(512, 1)



In [0]:
with open("inpu.txt", "rb") as fp:   
   inpu = pickle.load(fp)

In [0]:
input1 = inpu[0]
input1 = nn.Variable.from_numpy_array(input1)
print(input1.shape)

(1, 3, 256, 256)


In [0]:
output2 = D_basic(input1, use_wscale = False)

In [0]:
print(output2.d)

[[-3.3796804e+24]]
